In [2]:
import requests
import re
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from rusenttokenize import ru_sent_tokenize
from razdel import tokenize
from nltk.stem.snowball import SnowballStemmer
from collections import Counter
from nltk.corpus import stopwords

In [3]:
response = requests.get('https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/zhivago.txt')
data = response.text

## Задание 1

In [4]:
def clear(text):
    no_tags = re.sub(r'<[^>]+>', ' ', text)
    no_newline = re.sub(r'\n', '', no_tags) # тут будут проблемы со строками без пунктуации в конце типа заголовков, но ими мы пожертвуем:(
    no_nbrspaces = re.sub(r'\xa0', ' ', no_newline)
    no_spaces = re.sub('  +', ' ', no_nbrspaces)
    return no_spaces

In [5]:
cleared_data = clear(data)

## Задание 2

Я не поняла, как получится разделить текст на предложения, если мы уберем знаки пунктуации. Так что убирать пунктуацию будем только при токенизации, в предложениях она важна:

In [6]:
sent_tokenized = ru_sent_tokenize(cleared_data)

In [7]:
sents = [sent.lower() for sent in sent_tokenized]

In [8]:
punct = ['.',',','!','?',':',';','«','»','–', '-', '...', '(', ')', '„', '“']
tokens = [token.text.lower() for token in list(tokenize(cleared_data)) if token.text not in punct]

In [9]:
repeat_sents = [sent for sent in set(sents) if sents.count(sent) > 1]

1) Да, в тексте 132 таких предложения.

In [11]:
print(Counter([w for w in tokens if len(w) > 6]).most_common(1))

[('андреевич', 289)]


2) Это токен "андреевич".

## Задание 3

In [12]:
stemmer = SnowballStemmer('russian')

In [13]:
stems = [(word, stemmer.stem(word)) for word in tokens]

Попробуем найти возможные ошибки:

In [14]:
dif_words = []
no_stemming = []
for pair in set(stems):
    for pair1 in set(stems):
        if pair[0] != pair1[0] and pair[1] == pair1[1]:
            dif_words.append((pair, pair1))
    if pair[0] == pair[1]:
        no_stemming.append(pair)
    if len(dif_words) >=20 and len(no_stemming) >= 60:
        break     

1) "платы" и "платьями" (основа *плат-*), "шипит" и "шипах" (основа *шип-*), "окажемся" и "окажите" (основа *окаж-*)

2) "плывут", "нейдет", "сжав"

## Задание 4

In [15]:
print(stopwords.words('russian'))

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

Можно добавить туда больше вопросительных слов (*почему*, *откуда*), т.к. они обычно не несут особенной смысловой нагрузки, если только речь не идет о художественном приеме; еще там явно не хватает союза *который*. Имеет смысл занести в список вводные слова, которые нужны только для связности текста, например, *итак* или *во-первых*.

А еще не очень понятно, почему некоторые слова даны в списке только в начальной форме, а некоторые в разных. Если применять список к не-лемматизированным словам, то нужно дополнить многие парадигмы (*моего*, *кого*, *какого*).

## Задание 5

In [16]:
mystem = Mystem()
morph = MorphAnalyzer()

Installing mystem to C:\Users\Daria Arakelova/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.0-win7-64bit.zip


In [17]:
lemmas1 = [morph.parse(token)[0].normal_form for token in tokens]

In [30]:
lemmas2 = [l for l in mystem.lemmatize(cleared_data.lower()) if l.isalpha()]

Попробуем сравнить работу лемматизаторов на отрывке текста:

In [34]:
for i in range(150):
    print(lemmas1[i] + ' | ' + lemmas2[i])

борис | борис
леонид | леонидович
пастернак | пастернак
доктор | доктор
живаго | живаго
доктор | доктор
живаго | живаго
итоговый | итоговый
произведение | произведение
борис | борис
пастернак | пастернак
книга | книга
весь | весь
он | его
жизнь | жизнь
этот | этот
роман | роман
принести | приносить
он | он
автор | автор
мировой | мировой
известность | известность
и | и
нобелевский | нобелевский
премия | премия
присуждение | присуждение
который | который
обернуться | обертываться
для | для
поэт | поэт
оголтелый | оголтелый
политический | политический
травля | травля
обвинение | обвинение
в | в
измена | измена
родина | родина
и | и
в | в
результат | результат
стоить | стоить
он | он
жизнь | жизнь
доктор | доктор
живаго | живаго
роман | роман
сам | сам
ткань | ткань
который | который
убедительный | убедительно
свидетельствовать | свидетельствовать
о | о
чудо | чудо
чем | чем
весь | весь
размышление | размышление
доктор | доктор
и | и
обобщение | обобщение
автор | автор
человек | человек
к

По этому небольшому отрывку текста кажется, что mystem лучше справляется с задачей лемматизации. Вот его достоинства по сранению с pymorphy:
- Он правильно обрабатывает притяжятельное местоимение *его* (pymorphy приводит к личному *он*)
- Он нечувствителен к виду глагола, что можно считать достоинством, если мы хотим составить словарь текста. Хотя в каких-то других случаях, наверное, лучше выделять глаголы совершенного вида в самостоятельные леммы, как это делает pymorphy
- Он обрабатывает причастия как отдельные от глагола леммы, что справедливо с точки зрения их синтаксической дистрибуции
- Pymorphy считает, что *суть* -- это форма глагола *быть*, что очень мило, но непрактично

При этом mystem неправильно разрешает омонимию в слове *стоит*. В целом кажется, что анализ mystem-а не такой глубокий, как у pymorphy, но он лучше для каких-то прикладных задач.